<a href="https://colab.research.google.com/github/Ak-Gautam/efficient_llm_fine_tunes/blob/main/data_manipulation/Dataset_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub -q
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the dataset (considering it fits in memory, adjust if not)
dataset = load_dataset("teknium/OpenHermes-2.5", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1001551 [00:00<?, ? examples/s]

In [ ]:
source_in = ["metamath", "airoboros2.2"]

In [ ]:
dataset_trim = dataset.filter(lambda example: example['source'] in source_in)
dataset_trim

Filter:   0%|          | 0/1001551 [00:00<?, ? examples/s]

Dataset({
    features: ['custom_instruction', 'topic', 'model_name', 'model', 'skip_prompt_formatting', 'category', 'conversations', 'views', 'language', 'id', 'title', 'idx', 'hash', 'avatarUrl', 'system_prompt', 'source'],
    num_rows: 91828
})

In [ ]:
set(dataset_trim["source"])

{'airoboros2.2', 'metamath'}

In [ ]:
# Shuffle the dataset with a seed for reproducibility
seed = 42
shuffled_dataset = dataset.shuffle(seed=seed)

# Select the first 250,000 rows
sampled_dataset = shuffled_dataset.select(range(250000))

In [ ]:
!huggingface-cli login

In [ ]:
sampled_dataset.push_to_hub("glouriousgautam/openhermes250k")

### Fineweb

In [ ]:
!pip install huggingface_hub -q
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from datasets import load_dataset
import time

In [18]:
start_time = time.time()
# Load the dataset partially and filter it based on the token_count column
dataset = load_dataset("HuggingFaceFW/fineweb", split='train', streaming=True, columns=["text", "token_count"])
filtered_dataset = dataset.filter(lambda x: x["token_count"] > 1000)

# Initialize an empty list to store the chunks
chunks = []

# Iterate over the filtered dataset in chunks
chunk_size = 10000  # Adjust this value based on your available memory
for i in range(0, 90000, chunk_size):
    chunk = filtered_dataset.skip(i).take(chunk_size)
    # Append the chunk to the list of chunks
    chunks.append(chunk)

end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.2f} seconds")

Resolving data files:   0%|          | 0/23781 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23781 [00:00<?, ?it/s]

Elapsed time: 13.90 seconds


In [19]:
len(chunks)

9

In [20]:
from datasets import concatenate_datasets

# Concatenate the chunks into a single dataset
start_time = time.time()
full_dataset = concatenate_datasets(chunks)
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.2f} seconds")

Elapsed time: 204.66 seconds


In [ ]:
30k - 30,30
50k - 20, 58
60k - 20,85
90k - 14, 205
120k - 15,350
240k - 12, 1465


NameError: name 'full_dataset' is not defined